This markdown file is used to preprocess the data starting with the raw data output from the scrapers

In [40]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine.base import Engine
import plotly.express as px

# Import the data from the postgres db

In [2]:
def get_db_engine(
    username: str,
    password: str,
    protocol: str = "postgresql",
    server: str = "localhost",
    port: int = 5432,
    dbname: str = "ufc",
) -> Engine:

    engine = create_engine(
        f"{protocol}://" f"{username}:" f"{password}@" f"{server}:" f"{port}/" f"{dbname}",
        isolation_level="AUTOCOMMIT",
    )
    return engine

In [3]:
db_engine = get_db_engine('postgres', 'postgres', dbname='ufc')

In [83]:
with db_engine.connect() as conn:
    bouts = pd.read_sql('SELECT * FROM ufc.bouts', con = conn)
    fighters = pd.read_sql('SELECT * FROM ufc.fighters', con = conn)

## Data pre processing

In [84]:
# First convert bouts date to datetime
bouts['event_date'] = pd.to_datetime(bouts['event_date'], format="%B %d %Y")

In [85]:
fighters.head()


,index,id,fighter_name,fighter_record,height,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg
0,0,1,Hunter Azure,Record: 9-2-0,"5' 8""",145.0,69.0,Orthodox,Mar 02 1992,3.92,1.97,53,34,2.08,67,58,1.3
1,1,2,Michael Byrnes,Record: 5-3-0,"5' 11""",155.0,NaN,None,Jul 17 1990,0.00,0.00,0,0,0.00,0,0,0.0
2,2,3,Gleidson Cutis,Record: 7-4-0,"5' 9""",155.0,NaN,Orthodox,Feb 07 1989,2.99,0.00,52,0,8.28,0,59,0.0
3,3,4,Rolando Dy,Record: 9-7-1 (1 NC),"5' 8""",145.0,69.0,Orthodox,Aug 11 1990,3.04,0.30,37,20,4.47,68,52,0.0
4,4,5,Jessica Eye,Record: 15-11-0 (1 NC),"5' 6""",125.0,66.0,Orthodox,Jul 27 1986,3.86,0.58,37,40,4.19,57,55,0.5


In [86]:
# Filter out the fighters who did not have any fights yet
fighters_that_fought = set(bouts.fighter1).union(set(bouts.fighter2))
fighters = fighters.loc[fighters.fighter_name.isin(fighters_that_fought)]


In [87]:
# Extract total wins and losses
fighters['total_wins'] = fighters.fighter_record.map(lambda x: x.split('Record: ')[1].split('-')[0])
fighters['total_losses'] = fighters.fighter_record.map(lambda x: x.split('Record: ')[1].split('-')[1])
fighters['total_draws'] = fighters.fighter_record.map(lambda x: x.split('Record: ')[1].split('-')[2].split('(')[0])

# Drop fighter_record and index columns
fighters = fighters.drop(['fighter_record'], axis = 1)

In [88]:
# Clean date column
fighters.date_of_birth = fighters.date_of_birth.replace('--', None)
fighters['date_of_birth'] = pd.to_datetime(fighters['date_of_birth'], format="%b %d %Y")

In [90]:
# Drop na values for height
fighters = fighters.dropna(subset=['height'])

# Turn height into centimeters
fighters.height = fighters.height.replace('--', None)

fighters['height_feet'] = fighters.height.map(lambda x: int(x.split("' ")[0]))
fighters['height_inch'] = fighters.height.map(lambda x: int(x.split("' ")[1].replace('"', "")))
fighters['height_cm'] = 30.48 * fighters['height_feet'] + 2.54 * fighters['height_inch']
fighters = fighters.drop(['height', 'height_feet', 'height_inch'], axis = 1)

In [91]:
# check if there are fighters with the same name
fighters[fighters.duplicated(subset="fighter_name", keep=False)]

,index,id,fighter_name,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg,total_wins,total_losses,total_draws,height_cm
632,632,633,Michael McDonald,135.0,70.0,Orthodox,1991-01-15,2.69,1.09,42,66,2.76,52,57,1.4,17,4,0,175.26
634,634,635,Michael McDonald,205.0,NaN,Orthodox,1965-02-06,0.00,0.00,0,0,0.40,0,50,0.0,1,1,0,180.34
1593,1592,1593,Joey Gomez,155.0,71.0,Orthodox,1989-08-29,3.73,2.00,49,28,3.33,0,50,0.0,7,1,0,177.80
1595,1594,1595,Joey Gomez,135.0,73.0,Orthodox,1986-07-21,2.44,0.62,28,100,4.46,50,55,0.0,6,2,0,177.80
3484,3483,3484,Bruno Silva,185.0,74.0,Orthodox,1989-07-13,4.31,0.66,48,18,4.58,71,44,0.0,22,8,0,182.88
3485,3484,3485,Bruno Silva,125.0,65.0,Orthodox,1990-03-16,2.98,2.89,46,31,3.23,64,58,0.0,12,5,2,162.56


In [92]:
# note that we have several fighters who have the same names
# Fortunately, they belong to different weight classes
fighters.loc[(fighters.fighter_name=='Michael McDonald') & (fighters.weight==205), "fighter_name"] = 'Michael McDonald 205'
fighters.loc[(fighters.fighter_name=='Joey Gomez') & (fighters.weight==155), "fighter_name"] = 'Joey Gomez 155'
fighters.loc[(fighters.fighter_name=='Mike Davis') & (fighters.weight==145), "fighter_name"] = 'Mike Davis 145'
fighters.loc[(fighters.fighter_name=='Bruno Silva') & (fighters.weight==205), "fighter_name"] = 'Bruno Silva 125'

In [93]:
# Some fighters do not have statistics available, and we will remove those fighters.
fighers = fighters.loc[~((fighters["slpm"] == 0) &
                               (fighters["strike_acc"] == 0) & 
                               (fighters["sapm"] == 0) &
                               (fighters["strike_def"] == 0) &
                               (fighters["td_avg"] == 0) &
                               (fighters["td_acc"] == 0) &
                               (fighters["td_def"] == 0) &
                               (fighters["sub_avg"] == 0))].copy()  
                            

In [94]:
fighters = fighters.loc[fighters['date_of_birth']!='--',:].copy()
fighters.date_of_birth = pd.to_datetime(fighters.date_of_birth)
fighters.head()


,index,id,fighter_name,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg,total_wins,total_losses,total_draws,height_cm
0,0,1,Hunter Azure,145.0,69.0,Orthodox,1992-03-02,3.92,1.97,53,34,2.08,67,58,1.3,9,2,0,172.72
3,3,4,Rolando Dy,145.0,69.0,Orthodox,1990-08-11,3.04,0.30,37,20,4.47,68,52,0.0,9,7,1,172.72
4,4,5,Jessica Eye,125.0,66.0,Orthodox,1986-07-27,3.86,0.58,37,40,4.19,57,55,0.5,15,11,0,167.64
6,6,7,Mike Guymon,170.0,74.0,Orthodox,1974-09-17,2.75,1.87,65,37,0.79,54,66,1.2,15,6,1,182.88
9,9,10,Cristiane Justino,145.0,68.0,Orthodox,1985-07-09,7.28,0.66,52,55,2.25,94,64,0.4,21,2,0,172.72


In [95]:
# Get the fighters record in the ufc
def get_ufc_fights(fighter, bouts):
    """Extracts the total number of fights fought in the ufc"""
    bouts_test = bouts.loc[(bouts.fighter1 == fighter) | (bouts.fighter2 == fighter), :].copy()
    wins = len(bouts_test.loc[(bouts_test.winner == fighter) & (bouts_test.win == True), :].copy())
    losses = len(bouts_test.loc[(bouts_test.winner != fighter) & (bouts_test.win == True), :].copy())
    nc = len(bouts_test) - wins - losses
    return wins, losses, nc

fighters[['ufc_wins', 'ufc_losses', 'ufc_nc']] = [get_ufc_fights(fighter, bouts) for fighter in fighters.fighter_name]

In [96]:
# Remove the index, id and stance columns
fighters = fighters.drop(['index', 'id', 'stance', 'weight'], axis = 1)

In [97]:
# convert all dtypes except fighter name and date of birth to float
columns_to_float = fighters.columns[1:]
columns_to_float = columns_to_float.drop(['date_of_birth'])
fighters[columns_to_float] = fighters[columns_to_float].astype(float)

# Now we clean the bouts and merge the two together

In [98]:
# First drop all the fights without a winner from the dataset
bouts = bouts.loc[bouts.win == True, :].copy()
bouts.head()

,index,id,event_name,event_date,win,winner,fighter1,fighter2,weight_class,title_fight,...,sig_distance_attempted_1,sig_distance_attempted_2,sig_clinch_landed_1,sig_clinch_landed_2,sig_clinch_attempted_1,sig_clinch_attempted_2,sig_ground_landed_1,sig_ground_landed_2,sig_ground_attempted_1,sig_ground_attempted_2
0,0,1,UFC Fight Night: Dos Anjos vs. Fiziev,2022-07-09,True,Saidyokub Kakhramonov,Ronnie Lawrence,Saidyokub Kakhramonov,Bantamweight Bout,False,...,32.0,20.0,1.0,7.0,1.0,9.0,0.0,21.0,0.0,30.0
1,1,2,UFC Fight Night: Ortega vs. Rodriguez,2022-07-16,True,Emily Ducote,Jessica Penne,Emily Ducote,Women's Strawweight Bout,False,...,234.0,203.0,8.0,1.0,10.0,2.0,0.0,0.0,0.0,0.0
2,2,3,UFC Fight Night: Blaydes vs. Aspinall,2022-07-23,True,Nicolas Dalby,Claudio Silva,Nicolas Dalby,Welterweight Bout,False,...,20.0,59.0,0.0,16.0,1.0,21.0,0.0,8.0,0.0,14.0
3,3,4,UFC 277: Pena vs. Nunes 2,2022-07-30,True,Orion Cosce,Orion Cosce,Blood Diamond,Welterweight Bout,False,...,27.0,60.0,9.0,29.0,14.0,37.0,8.0,0.0,11.0,1.0
4,4,5,UFC Fight Night: Santos vs. Hill,2022-08-06,True,Mayra Bueno Silva,Mayra Bueno Silva,Stephanie Egger,Women's Bantamweight Bout,False,...,4.0,7.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0


In [99]:
# Filter out relevant columns from the bouts df
bouts = bouts[['fighter1', 'fighter2', 'winner']].copy()

In [100]:
# Create a loser column which will be equal to the fighter who lost the bout
bouts['loser'] = bouts.apply(lambda x: x['fighter1'] if x['fighter2'] == x['winner'] else x['fighter2'], axis = 1)

# drop fighter1 and fighter2 columns
bouts = bouts.drop(['fighter1', 'fighter2'], axis = 1)

# randomly distribute the winner and the loser columns over the fighter1 and fighter2 columns
bouts['fighter1'] = bouts.apply(lambda x: x['winner'] if np.random.rand() > 0.5 else x['loser'], axis = 1)
bouts['fighter2'] = bouts.apply(lambda x: x['winner'] if x['fighter1'] == x['loser'] else x['loser'], axis = 1)

# set the win column to 1 if the fighter1 is the winner and 0 otherwise
bouts['win'] = bouts.apply(lambda x: 1 if x['fighter1'] == x['winner'] else 0, axis = 1)

# reorder the columns to be fighter1, fighter2, win
bouts = bouts[['fighter1', 'fighter2', 'win']].copy()

bouts.head()

,fighter1,fighter2,win
0,Saidyokub Kakhramonov,Ronnie Lawrence,1
1,Emily Ducote,Jessica Penne,1
2,Nicolas Dalby,Claudio Silva,1
3,Orion Cosce,Blood Diamond,1
4,Mayra Bueno Silva,Stephanie Egger,1


In [101]:
# Quick check to see if the positive case for our model occurs roughly 50% of the time
sum(bouts.win)/len(bouts)


0.5085000752219047

In [108]:
bouts = bouts[['fighter1', 'fighter2', 'win']].copy()

# Merge the bouts dataframe with the difference in statistics between the two fighters
bouts = bouts.merge(fighters, left_on='fighter1', right_on='fighter_name', how='left')
bouts = bouts.merge(fighters, left_on='fighter2', right_on='fighter_name', how='left', suffixes=('_fighter1', '_fighter2'))

# Calculate the difference in statistics between the two fighters
bouts['reach_diff'] = bouts['reach_fighter1'] - bouts['reach_fighter2']
bouts['height_diff'] = bouts['height_cm_fighter1'] - bouts['height_cm_fighter2']
bouts['age_diff'] = (pd.to_datetime(bouts['date_of_birth_fighter1']) - pd.to_datetime(bouts['date_of_birth_fighter2'])).dt.days / 365.25
bouts['slpm_diff'] = bouts['slpm_fighter1'] - bouts['slpm_fighter2']
bouts['td_avg_diff'] = bouts['td_avg_fighter1'] - bouts['td_avg_fighter2']
bouts['strike_acc_diff'] = bouts['strike_acc_fighter1'] - bouts['strike_acc_fighter2']
bouts['td_acc_diff'] = bouts['td_acc_fighter1'] - bouts['td_acc_fighter2']
bouts['sapm_diff'] = bouts['sapm_fighter1'] - bouts['sapm_fighter2']
bouts['td_def_diff'] = bouts['td_def_fighter1'] - bouts['td_def_fighter2']
bouts['strike_def_diff'] = bouts['strike_def_fighter1'] - bouts['strike_def_fighter2']
bouts['sub_avg_diff'] = bouts['sub_avg_fighter1'] - bouts['sub_avg_fighter2']
bouts['total_wins_diff'] = bouts['total_wins_fighter1'] - bouts['total_wins_fighter2']
bouts['total_losses_diff'] = bouts['total_losses_fighter1'] - bouts['total_losses_fighter2']
bouts['total_draws_diff'] = bouts['total_draws_fighter1'] - bouts['total_draws_fighter2']
bouts['ufc_wins_diff'] = bouts['ufc_wins_fighter1'] - bouts['ufc_wins_fighter2']
bouts['ufc_losses_diff'] = bouts['ufc_losses_fighter1'] - bouts['ufc_losses_fighter2']
bouts['ufc_nc_diff'] = bouts['ufc_nc_fighter1'] - bouts['ufc_nc_fighter2']

# Drop the columns that we will not use for our model
bouts.drop(['fighter_name_fighter1', 'reach_fighter1', 'date_of_birth_fighter1', 'slpm_fighter1', 'td_avg_fighter1', 'strike_acc_fighter1', 'td_acc_fighter1', 'sapm_fighter1', 'td_def_fighter1', 'strike_def_fighter1', 'sub_avg_fighter1', 'total_wins_fighter1', 'total_losses_fighter1', 'total_draws_fighter1', 'height_cm_fighter1', 'ufc_wins_fighter1', 'ufc_losses_fighter1', 'ufc_nc_fighter1', 'fighter_name_fighter2', 'reach_fighter2', 'date_of_birth_fighter2', 'slpm_fighter2', 'td_avg_fighter2', 'strike_acc_fighter2', 'td_acc_fighter2', 'sapm_fighter2', 'td_def_fighter2', 'strike_def_fighter2', 'sub_avg_fighter2', 'total_wins_fighter2', 'total_losses_fighter2', 'total_draws_fighter2', 'height_cm_fighter2', 'ufc_wins_fighter2', 'ufc_losses_fighter2', 'ufc_nc_fighter2'], axis=1, inplace=True)

# Drop all nan values for now
bouts = bouts.dropna()

bouts.head()

,fighter1,fighter2,win,reach_diff,height_diff,age_diff,slpm_diff,td_avg_diff,strike_acc_diff,td_acc_diff,sapm_diff,td_def_diff,strike_def_diff,sub_avg_diff,total_wins_diff,total_losses_diff,total_draws_diff,ufc_wins_diff,ufc_losses_diff,ufc_nc_diff
0,Saidyokub Kakhramonov,Ronnie Lawrence,1,1.0,0.00,3.419576,-0.59,-0.96,-2.0,-25.0,-0.84,33.0,4.0,1.7,2.0,0.0,0.0,0.0,-1.0,0.0
1,Emily Ducote,Jessica Penne,1,-4.0,-7.62,10.921287,5.11,-1.39,24.0,-21.0,-0.54,60.0,27.0,-0.6,-2.0,-1.0,0.0,-2.0,-4.0,0.0
2,Nicolas Dalby,Claudio Silva,1,3.0,0.00,2.195756,0.74,-1.63,-3.0,5.0,0.22,21.0,7.0,-1.0,6.0,0.0,1.0,-1.0,0.0,2.0
3,Orion Cosce,Blood Diamond,1,-5.0,0.00,6.012320,-0.34,2.30,-10.0,40.0,1.49,3.0,3.0,0.0,5.0,-1.0,0.0,1.0,-1.0,0.0
4,Mayra Bueno Silva,Stephanie Egger,1,-2.0,0.00,3.025325,2.33,-3.29,1.0,-46.0,3.26,34.0,7.0,0.9,2.0,-1.0,1.0,2.0,0.0,1.0


In [109]:
# Write the bouts df to the database
bouts.to_sql('model_input', db_engine, schema='ufc', if_exists='replace', index=False)


625

In [110]:
# Get nans of each column
bouts.isna().sum()

fighter1             0
fighter2             0
win                  0
reach_diff           0
height_diff          0
age_diff             0
slpm_diff            0
td_avg_diff          0
strike_acc_diff      0
td_acc_diff          0
sapm_diff            0
td_def_diff          0
strike_def_diff      0
sub_avg_diff         0
total_wins_diff      0
total_losses_diff    0
total_draws_diff     0
ufc_wins_diff        0
ufc_losses_diff      0
ufc_nc_diff          0
dtype: int64

## Potential questions for data analysis
Is championship experience a real thing?! win rates when more championship fights
Ring rust
Progression of stopages over time
Duration of bouts over time


